In [4]:
import numpy as np
import polars as pl
from tqdm import tqdm

from typing import List, Any

import scipy.sparse as sp
from sklearn.model_selection import train_test_split

import random
from collections import Counter

In [3]:
data = pl.read_parquet('train.parquet')
# датафрейм с обратными ребрами
data_rev = (
    data
    .rename({'uid': 'friend_uid', 'friend_uid': 'uid'})
    .select('uid', 'friend_uid')
)

# соединим все в один граф
data = pl.concat([data, data_rev])
data

uid,friend_uid
i64,i64
93464,114312
93464,103690
93464,108045
93464,116128
93464,94113
93464,101668
93464,118820
93464,93617
93464,97587


Данные состоят из двух колонок:

- `uid` – идентификатор пользователя
- `friend_uid` – идентификатор друга этого пользователя

Нашей задачей будет порекомендовать возможных друзей, для оценки вашего решения будет использоваться метрика Recall@10, равная проценту верно угаданных друзей

In [15]:
TOP_K = 20
RANDOM_STATE = 42

SUBMISSION_PATH = 'submission.parquet'


def user_intersection(y_rel: List[Any], y_rec: List[Any], k: int = 10) -> int:
    """
    :param y_rel: relevant items
    :param y_rec: recommended items
    :param k: number of top recommended items
    :return: number of items in intersection of y_rel and y_rec (truncated to top-K)
    """
    return len(set(y_rec[:k]).intersection(set(y_rel)))


def user_recall(y_rel: List[Any], y_rec: List[Any], k: int = 10) -> float:
    """
    :param y_rel: relevant items
    :param y_rec: recommended items
    :param k: number of top recommended items
    :return: percentage of found relevant items through recommendations
    """
    return user_intersection(y_rel, y_rec, k) / min(k, len(set(y_rel)))

## Валидация

Так как у нас нет временной последовательности и рекомендации друзей не так сильно зависят от временной составляющей, в качестве валидации можно использовать случайно выбранные ребра в графе (при этом для каждого пользователя будет равная пропорция друзей в валидации, которую можно достичь с помощью stratify параметра)

In [16]:
# зафиксируем генератор случайных чисел
random.seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)

In [6]:
# отфильтруем тех пользователей, у которых только один друг :(
# для того, чтобы в тренировочной выборке и валидации было хотя бы по одному другу
friends_count = data.groupby('uid').count()
filtered_uid = set(friends_count.filter(pl.col('count') > 1)['uid'].to_list())

data_filtered = data.filter(pl.col('uid').is_in(filtered_uid))

# случайно выбираем ребра для валидационной выборки
train_df, test_df = train_test_split(
    data_filtered.filter(pl.col('uid').is_in(filtered_uid)),
    stratify=data_filtered['uid'],
    test_size=0.1,
    random_state=RANDOM_STATE
)

train_df

uid,friend_uid
i64,i64
62053,63575
31895,59356
97127,32271
89,11703
105178,47188
116127,52662
33824,15235
23690,103992
94660,45709


## Бейзлайн (Random)

In [198]:
grouped_df = (
    test_df
    .groupby('uid')
    .agg(pl.col('friend_uid').alias('y_rel'))
    .join(
        train_df
        .groupby('uid')
        .agg(pl.col('friend_uid').alias('user_history')),
        'uid',
        how='left'
    )
)

median_seq_len = int(grouped_df['user_history'].apply(len).median())
print(f"среднее число uid в user_history: {median_seq_len}")

среднее число uid в user_history: 36


In [8]:
n_users = train_df['uid'].max() + 1

# количество друзей у каждого пользователя
friends_count = np.zeros(n_users)
for uid, count in Counter(train_df['uid']).items():
    friends_count[uid] = count
    
friends_count /= sum(friends_count)

In [9]:
recall_list = []
recs = np.random.choice(n_users, size=(n_users, TOP_K + median_seq_len), p=friends_count)

for user_id, y_rel, user_history in tqdm(grouped_df.rows()):
    y_rec = [uid for uid in recs[user_id] if uid not in user_history]
    recall_list.append(user_recall(y_rel, y_rec))
    
print(f'Recall@{TOP_K} = {np.mean(recall_list)}')

100%|██████████| 92562/92562 [00:08<00:00, 10456.45it/s]

Recall@10 = 0.0003110784946203369


## Построим рекомендации

In [34]:
# посчитаем вероятности уже по всем имеющимся данным
n_users = data['uid'].max() + 1

# количество друзей у каждого пользователя
friends_count = np.zeros(n_users)
for uid, count in Counter(data['uid']).items():
    friends_count[uid] = count
    
friends_count /= sum(friends_count)

In [11]:
sample_submission = pl.read_parquet('sample_submission.parquet')

grouped_df = (
    sample_submission.select('uid')
    .join(
        train_df
        .groupby('uid')
        .agg(pl.col('friend_uid').alias('user_history')),
        'uid',
        how='left'
    )
)

submission = []
recs = np.random.choice(n_users, size=(n_users, TOP_K + median_seq_len), p=friends_count)

for user_id, user_history in tqdm(grouped_df.rows()):
    user_history = [] if user_history is None else user_history
    
    y_rec = [uid for uid in recs[user_id] if uid not in user_history]
    submission.append((user_id, y_rec))
    
submission = pl.DataFrame(submission, schema=['user_id', 'y_recs'])
submission.write_parquet('submission.parquet')
submission

100%|██████████| 85483/85483 [00:08<00:00, 9611.88it/s] 


user_id,y_recs
i64,list[i64]
0,"[75174, 40482, … 107746]"
1,"[27663, 82181, … 37095]"
3,"[105454, 12906, … 43868]"
4,"[2627, 60169, … 108457]"
5,"[29164, 53357, … 33803]"
6,"[62015, 44506, … 3835]"
7,"[74067, 79534, … 53438]"
8,"[75913, 31803, … 32356]"
9,"[26079, 73565, … 111629]"


# Начинаем выполнять задание

Дано:
- информация о связях между пользователями

Надо:
- составить эмбеддинги пользователей
- для каждого пользователя найти наиболее похожих других пользователей, не являющихся сейчас друзьями

## Составляем эмбеддинги пользователей

In [15]:
train_df

uid,friend_uid
i64,i64
62053,63575
31895,59356
97127,32271
89,11703
105178,47188
116127,52662
33824,15235
23690,103992
94660,45709


In [28]:
import torch
from torch_geometric.data import Data, HeteroData
from torch_geometric.nn import Node2Vec, SAGEConv, LightGCN, to_hetero
import pickle
from sklearn.metrics.pairwise import cosine_distances

/home/jupyter-pavel.lyubovin/.conda/envs/recsys/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [29]:
# будем использовать cuda, если доступны вычисления на gpu
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'device: {device}')

device: cpu


In [24]:
# Не знаю, как лучше кодировать вершины, пусть будут просто нули
x = torch.tensor([0] * n_users, dtype=torch.float)

# Ребра
train_df
edge_index = torch.tensor([train_df['uid'].to_numpy(),
                           train_df['friend_uid'].to_numpy()], dtype=torch.long)

# Формирую граф
torch_data = Data(x=x, edge_index=edge_index)#.T.contiguous())
torch_data

Data(x=[120061], edge_index=[2, 5166881])

In [28]:
model = Node2Vec(
    torch_data.edge_index,
    embedding_dim=128,  # размер эмбеддинга вершины
    walk_length=20,  # длина случайного блуждания
    context_size=10,  # размер окна из случайного блуждания (как в w2v)
    walks_per_node=10,  # количество случайных блужданий из одной вершины
    num_negative_samples=1,  # количество негативных примеров на один позитивный
    p=1.0,  # параметр вероятности вернуться в предыдущую вершину
    q=1.0,  # параметр вероятности исследовать граф вглубь
    sparse=True,
).to(device)

# класс Node2Vec предоставляет сразу генератор случайного блуждания
loader = model.loader(batch_size=512, shuffle=True, num_workers=4)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)


def train():
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
        # pos_rw – последовательность из случайного блуждания
        # neg_rw – случайные негативные примеры
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)


# @torch.no_grad()
# def test():
#     model.eval()
#     z = model()
#     acc = model.test(
#         train_z=z[data.train_mask],
#         train_y=data.y[data.train_mask],
#         test_z=z[data.test_mask],
#         test_y=data.y[data.test_mask],
#         max_iter=150,
#     )
#     return acc


for epoch in range(1, 101):
    loss = train()
    # acc = test()
    if epoch % 10 == 0:
        # print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Acc: {acc:.4f}')
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')


# @torch.no_grad()
# def plot_points(colors):
#     model.eval()
#     z = model().cpu().numpy()
#     z = TSNE(n_components=2).fit_transform(z)
#     y = data.y.cpu().numpy()

#     plt.figure(figsize=(8, 8))
#     for i in range(dataset.num_classes):
#         plt.scatter(z[y == i, 0], z[y == i, 1], s=20, color=colors[i])
#     plt.axis('off')
#     plt.show()


# colors = [
#     '#ffc0cb', '#bada55', '#008080', '#420420', '#7fe5f0', '#065535', '#ffd700'
# ]
# plot_points(colors)

Epoch: 010, Loss: 1.0625
Epoch: 020, Loss: 1.0559
Epoch: 030, Loss: 1.0549
Epoch: 040, Loss: 1.0546
Epoch: 050, Loss: 1.0538
Epoch: 060, Loss: 1.0537
Epoch: 070, Loss: 1.0538
Epoch: 080, Loss: 1.0534
Epoch: 090, Loss: 1.0533
Epoch: 100, Loss: 1.0534


In [30]:
model.eval()

# Сохранение модели
with open('model_node2vec.pkl', 'wb') as handle:
    pickle.dump(model, handle)

In [31]:
# Чтение модели
with open('model_node2vec.pkl', 'rb') as handle:
    model_node2vec = pickle.load(handle)

In [33]:
# Предполагаю, что вот так можно вытащить все эмбеддинги всех клиентов
model_node2vec.eval()
z = model_node2vec().cpu().detach().numpy()
z

array([[ 0.2795245 ,  0.05894143, -0.06608918, ..., -0.20425828,
         0.10098294, -0.01275784],
       [-0.28717944,  0.21700333,  0.13922796, ..., -0.2867469 ,
        -0.2612505 , -0.2065052 ],
       [-0.5303003 , -0.2508898 ,  0.17089486, ..., -0.01172561,
         0.06361678, -0.11384055],
       ...,
       [-0.13796169, -0.07290526, -0.10769265, ...,  0.01244585,
         0.08405679,  0.09053381],
       [-0.23562124, -0.10650256,  0.16088043, ...,  0.26392296,
        -0.3374337 , -0.12853004],
       [-0.39085642,  0.2202326 ,  0.18887302, ..., -0.06343409,
         0.22059086,  0.26544124]], dtype=float32)

In [34]:
z.shape

(120061, 128)

In [48]:
sample_submission = pl.read_parquet('sample_submission.parquet')

grouped_df = (
    test_df
    .groupby('uid')
    .agg(pl.col('friend_uid').alias('y_rel'))
    .join(
        train_df
        .groupby('uid')
        .agg(pl.col('friend_uid').alias('user_history')),
        'uid',
        how='left'
    )
)

submission = []

grouped_df_pandas = grouped_df.to_pandas()

In [50]:
recall_list = []

CHUNK = 1000
for i in tqdm(range(121)):
    distance = cosine_distances(z[i * CHUNK : (i + 1) * CHUNK], z)

    recs = distance.argsort()
    recs = recs[:, :TOP_K + median_seq_len]

    for user_id in range(i * CHUNK, min((i + 1) * CHUNK, z.shape[0])):
        df = grouped_df_pandas[grouped_df_pandas['uid'] == user_id]
        if df.shape[0]:
            user_history = df['user_history'].values[0]
            y_rel = df['y_rel'].values[0]
        # Если нет такого юзера в grouped_df_pandas или его история is None
        if user_history is None:
            user_history = []
            y_rel = []
        # Добавляю в user_history id самого пользователя, чтоы потом его отфильтровать
        user_history = np.concatenate([user_history, [user_id]])
        
        y_rec = [uid for uid in recs[user_id - i * CHUNK] if uid not in user_history]
        submission.append((user_id, y_rec))

        recall_list.append(user_recall(y_rel, y_rec))

print(f'Recall@{TOP_K} = {np.mean(recall_list)}') 


In [227]:
submission = pl.DataFrame(submission, schema=['user_id', 'y_recs'])
submission.write_parquet('submission.parquet')
submission

user_id,y_recs
i64,list[i64]
0,"[79793, 71727, … 38370]"
1,"[52180, 78660, … 14676]"
2,"[60344, 24024, … 61973]"
0,"[79793, 71727, … 38370]"
1,"[52180, 78660, … 14676]"
2,"[60344, 24024, … 61973]"
3,"[55591, 109130, … 37287]"
4,"[57778, 82171, … 68721]"
5,"[100609, 33398, … 18562]"


Результат:

Recall@20 чкть менее 6%.

А надо 18%

## Попробую подобрать параметры для Node2Vec с помощью Оптуны на маленьком датасете

Если не получится, то буду искать ответы за пределами данных курса

In [84]:
data = pl.read_parquet('train.parquet')

# Делаю подвыборку 10_000 юзеров
# N_USERS_IN_PART = 10_000
N_USERS_IN_PART = data.shape[0]

data = data.filter((pl.col('uid') < N_USERS_IN_PART) & (pl.col('friend_uid') < N_USERS_IN_PART))

# датафрейм с обратными ребрами
data_rev = (
    data
    .rename({'uid': 'friend_uid', 'friend_uid': 'uid'})
    .select('uid', 'friend_uid')
)

# соединим все в один граф
data = pl.concat([data, data_rev])
data

uid,friend_uid
i64,i64
93464,114312
93464,103690
93464,108045
93464,116128
93464,94113
93464,101668
93464,118820
93464,93617
93464,97587


In [85]:
# отфильтруем тех пользователей, у которых только один друг :(
# для того, чтобы в тренировочной выборке и валидации было хотя бы по одному другу
friends_count = data.groupby('uid').count()
filtered_uid = set(friends_count.filter(pl.col('count') > 1)['uid'].to_list())

data_filtered = data.filter(pl.col('uid').is_in(filtered_uid))

# случайно выбираем ребра для валидационной выборки
train_df, test_df = train_test_split(
    data_filtered,#.filter(pl.col('uid').is_in(filtered_uid)),
    stratify=data_filtered['uid'],
    test_size=0.1,
    random_state=RANDOM_STATE
)

train_df

uid,friend_uid
i64,i64
62053,63575
31895,59356
97127,32271
89,11703
105178,47188
116127,52662
33824,15235
23690,103992
94660,45709


In [86]:
# посчитаем вероятности уже по всем имеющимся данным
n_users = data['uid'].max() + 1

# количество друзей у каждого пользователя
friends_count = np.zeros(n_users)
for uid, count in Counter(data['uid']).items():
    friends_count[uid] = count
    
friends_count /= sum(friends_count)

In [87]:
# Не знаю, как лучше кодировать вершины, пусть будут просто нули
x = torch.tensor([0] * n_users, dtype=torch.float)

# Ребра
edge_index = torch.tensor([train_df['uid'].to_numpy(),
                           train_df['friend_uid'].to_numpy()], dtype=torch.long)

# Формирую граф
torch_data = Data(x=x, edge_index=edge_index)#.T.contiguous())
torch_data

Data(x=[120061], edge_index=[2, 5166881])

In [88]:
grouped_df = (
    test_df
    .groupby('uid')
    .agg(pl.col('friend_uid').alias('y_rel'))
    .join(
        train_df
        .groupby('uid')
        .agg(pl.col('friend_uid').alias('user_history')),
        'uid',
        how='left'
    )
)
grouped_df_pandas = grouped_df.to_pandas()

# Костыль
median_seq_len = 36

In [89]:
grouped_df_pandas

,uid,y_rel,user_history
0,86624,"[26828, 10788]","[12811, 9091, 56344, 39979, 26230, 30958, 1370..."
1,76608,[111287],"[76584, 81295, 15784, 8507, 1338]"
2,58688,"[111655, 111579, 54258, 79169, 44950, 113027, ...","[9763, 50117, 40601, 19973, 83101, 16136, 9023..."
3,9888,"[39233, 60668, 26779, 15052]","[20177, 65780, 101579, 66047, 25379, 30505, 88..."
4,101472,"[73966, 97168, 5190, 32552, 10425, 35435, 2427...","[35572, 49060, 102361, 2765, 90929, 99688, 128..."
...,...,...,...
92557,87615,"[51709, 103014, 35561, 41273, 2767, 4850, 8573...","[73882, 84648, 106415, 38173, 43941, 75240, 38..."
92558,54847,"[33784, 43352, 61057, 112843, 28105]","[55435, 59127, 75121, 117966, 67600, 80025, 65..."
92559,30815,"[37464, 70856, 4578, 117867, 80306, 45387, 102...","[84619, 13683, 104100, 105147, 48529, 25596, 3..."
92560,70047,[37000],"[85071, 36150, 57605, 56935, 23581, 94278, 221..."


In [91]:
model = Node2Vec(
    torch_data.edge_index,
    embedding_dim=128,  # размер эмбеддинга вершины
    walk_length=20,  # длина случайного блуждания
    context_size=10,  # размер окна из случайного блуждания (как в w2v)
    walks_per_node=10,  # количество случайных блужданий из одной вершины
    num_negative_samples=1,  # количество негативных примеров на один позитивный
    p=1.0,  # параметр вероятности вернуться в предыдущую вершину
    q=1.0,  # параметр вероятности исследовать граф вглубь
    sparse=True,
).to(device)

# класс Node2Vec предоставляет сразу генератор случайного блуждания
loader = model.loader(batch_size=512, shuffle=True, num_workers=32)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)


def train():
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
        # pos_rw – последовательность из случайного блуждания
        # neg_rw – случайные негативные примеры
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)


@torch.no_grad()
def test():
    
    model.eval()
    # z = model()
    # Предполагаю, что вот так можно вытащить все эмбеддинги всех клиентов
    z = model().cpu().detach().numpy()

    recall_list = []

    CHUNK = 1000
    for i in range(((z.shape[0] - 1) // CHUNK) + 1):
        distance = cosine_distances(z[i * CHUNK : (i + 1) * CHUNK], z)

        recs = distance.argsort()
        recs = recs[:, :TOP_K + median_seq_len]

        for user_id in range(i * CHUNK, min((i + 1) * CHUNK, z.shape[0])):
            df = grouped_df_pandas[grouped_df_pandas['uid'] == user_id]
            if df.shape[0]:
                user_history = df['user_history'].values[0]
                y_rel = df['y_rel'].values[0]
            # Если нет такого юзера в grouped_df_pandas или его история is None
            else:
                user_history = []
                y_rel = []
            # Добавляю в user_history id самого пользователя, чтоы потом его отфильтровать
            user_history = np.concatenate([user_history, [user_id]])
            
            y_rec = [uid for uid in recs[user_id - i * CHUNK] if uid not in user_history]
            # submission.append((user_id, y_rec))

            # print(f"{y_rel = }, {y_rec = }")

            if len(y_rel):
                recall_list.append(user_recall(y_rel, y_rec, k=20))

    # print(f'Recall@{TOP_K} = {np.mean(recall_list)}') 


    return np.mean(recall_list)


for epoch in tqdm(range(1, 101)):
    loss = train()
    
    if epoch % 10 == 0:
        recall_20 = test()
        # print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Acc: {acc:.4f}')
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Recall@20: {recall_20:.4f}')


 10%|█         | 10/100 [1:26:35<19:47:30, 791.67s/it]

Epoch: 010, Loss: 1.0629, Recall@20: 0.0840


 20%|██        | 20/100 [2:54:07<17:54:52, 806.16s/it]

Epoch: 020, Loss: 1.0564, Recall@20: 0.1119


 30%|███       | 30/100 [4:19:54<15:35:47, 802.11s/it]

...

Epoch: 080, Loss: 1.0537, Recall@20: 0.0941


 90%|█████████ | 90/100 [13:05:02<2:11:48, 790.81s/it]

Epoch: 090, Loss: 1.0536, Recall@20: 0.0933


100%|██████████| 100/100 [14:31:34<00:00, 522.95s/it] 

Epoch: 100, Loss: 1.0535, Recall@20: 0.0922


In [92]:
sample_submission = pl.read_parquet('sample_submission.parquet')

submission = []

recall_list = []

z = model().cpu().detach().numpy()

CHUNK = 1000
for i in tqdm(range(121)):
    distance = cosine_distances(z[i * CHUNK : (i + 1) * CHUNK], z)

    recs = distance.argsort()
    recs = recs[:, :TOP_K + median_seq_len]

    for user_id in range(i * CHUNK, min((i + 1) * CHUNK, z.shape[0])):
        df = grouped_df_pandas[grouped_df_pandas['uid'] == user_id]
        if df.shape[0]:
            user_history = df['user_history'].values[0]
            y_rel = df['y_rel'].values[0]
        # Если нет такого юзера в grouped_df_pandas или его история is None
        if user_history is None:
            user_history = []
            y_rel = []
        # Добавляю в user_history id самого пользователя, чтоы потом его отфильтровать
        user_history = np.concatenate([user_history, [user_id]])
        
        y_rec = [uid for uid in recs[user_id - i * CHUNK] if uid not in user_history]
        submission.append((user_id, y_rec))

        recall_list.append(user_recall(y_rel, y_rec))

print(f'Recall@{TOP_K} = {np.mean(recall_list)}') 

submission = pl.DataFrame(submission, schema=['user_id', 'y_recs'])
submission.write_parquet('submission.parquet')
submission

100%|██████████| 121/121 [23:04<00:00, 11.44s/it]


Recall@20 = 0.049456344479915836


user_id,y_recs
i64,list[i64]
0,"[68034, 79051, … 64157]"
1,"[12009, 44749, … 118393]"
2,"[60598, 15467, … 115276]"
3,"[52256, 109130, … 116129]"
4,"[85427, 50616, … 20847]"
5,"[50151, 11322, … 60942]"
6,"[64142, 103138, … 84218]"
7,"[102537, 42645, … 81471]"
8,"[106972, 93246, … 865]"


In [97]:
sample_submission.to_pandas()

,uid,recs
0,0,"[59396, 3593, 91657, 39947, 69650, 106514, 261..."
1,1,"[118282, 109076, 69655, 101406, 59934, 99878, ..."
2,3,"[15364, 12807, 99340, 97296, 12818, 9750, 1743..."
3,4,"[69648, 95770, 50714, 26142, 17952, 1570, 1009..."
4,5,"[67591, 42002, 3603, 14882, 94246, 27174, 7210..."
...,...,...
85478,119383,"[15881, 94225, 87072, 8236, 85549, 41006, 5073..."
85479,119425,"[52743, 72713, 42011, 59424, 41504, 4141, 2001..."
85480,119457,"[58881, 14349, 67098, 54300, 110621, 19487, 18..."
85481,119486,"[6144, 33284, 59398, 92678, 3599, 15891, 34323..."


In [98]:
submission.to_pandas()

,user_id,y_recs
0,0,"[68034, 79051, 115056, 74766, 67322, 54883, 11..."
1,1,"[12009, 44749, 101936, 96208, 14884, 75484, 11..."
2,2,"[60598, 15467, 15411, 8915, 13920, 17648, 1057..."
3,3,"[52256, 109130, 99732, 94808, 70188, 99834, 10..."
4,4,"[85427, 50616, 57954, 18146, 8843, 60289, 1084..."
...,...,...
120056,120056,"[99696, 44039, 93766, 21271, 28377, 25668, 898..."
120057,120057,"[66183, 26622, 101018, 59781, 80366, 14712, 61..."
120058,120058,"[17851, 100331, 38897, 31044, 73082, 76397, 10..."
120059,120059,"[24642, 99233, 53574, 50035, 705, 86093, 11446..."


In [101]:
len(set(submission.to_pandas()['user_id']).intersection(set(sample_submission.to_pandas()['uid'])))

85483

1. Проверить пересечение uid в sample_submission и submission
2. Закинуть submission по последнему расчету
3. Посчитать тут на cpu 20 эпох (лучшее качество модели)
4. Запустить ноутбук на colabe
5. Попробовать оптимизировать модель там
6. Попробовать Adamic-Adam index
7. Посмотреть решение автора